In [1]:
import tensorflow as tf 

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [25]:
data_x, data_y = [], []
num_id = 9
img_h=100
img_w=100
img_c=3
img_length = 250*250*3

for i in range(num_id):
    part_x, part_y = [], []
    folder = 'pixiv/five_'+str(i)
    for img_file in os.listdir(folder):
#         print(folder+'/'+img_file)
        img = np.array(Image.open(folder+'/'+img_file))
#         img = np.reshape(img, len(img)*len(img[0])*len(img[0][0]))
        part_x.append(img/255.0)
        
        label = [0 for j in range(0, num_id)]
        label[i]=1
        part_y.append(label)   
        if(len(part_x)>=3800):
            break
    print(len(part_x))
    data_x.extend(part_x)
    data_y.extend(part_y)
    print(folder+' is completed.')

print(len(data_x))
print(len(data_y))

3800
pixiv/five_0 is completed.
3800
pixiv/five_1 is completed.
3800
pixiv/five_2 is completed.
3800
pixiv/five_3 is completed.
3800
pixiv/five_4 is completed.
3800
pixiv/five_5 is completed.
3800
pixiv/five_6 is completed.
3800
pixiv/five_7 is completed.
3800
pixiv/five_8 is completed.
34200
34200


In [26]:
data_x = np.array(data_x, dtype='float32')
data_y = np.array(data_y, dtype='float32')

print(data_x[:10])
print(data_y[:10])

[[[[ 0.09411765  0.07058824  0.07058824]
   [ 0.13725491  0.11372549  0.11372549]
   [ 0.16470589  0.14117648  0.14117648]
   ..., 
   [ 0.43529412  0.35686275  0.32941177]
   [ 0.44705883  0.35686275  0.33333334]
   [ 0.47450981  0.36470589  0.34901962]]

  [[ 0.10980392  0.08627451  0.08627451]
   [ 0.14509805  0.12156863  0.12156863]
   [ 0.16470589  0.14117648  0.14117648]
   ..., 
   [ 0.44313726  0.36078432  0.34117648]
   [ 0.4627451   0.36862746  0.35294119]
   [ 0.49411765  0.38431373  0.37254903]]

  [[ 0.1254902   0.10196079  0.10196079]
   [ 0.14901961  0.1254902   0.1254902 ]
   [ 0.16078432  0.13725491  0.13725491]
   ..., 
   [ 0.4509804   0.35686275  0.34117648]
   [ 0.47058824  0.37254903  0.35686275]
   [ 0.50588238  0.39607844  0.38431373]]

  ..., 
  [[ 0.21176471  0.20784314  0.19215687]
   [ 0.19607843  0.19215687  0.17254902]
   [ 0.19607843  0.19215687  0.17254902]
   ..., 
   [ 0.08627451  0.05882353  0.08627451]
   [ 0.08627451  0.05882353  0.08627451]
   [ 0.

In [27]:
order = [i for i in range(len(data_x))]
random.shuffle(order)

print(len(order))
n=int(len(data_x)*9/10)
print(n)

train_x = [data_x[i] for i in order[0:n]]
train_y = [data_y[i] for i in order[0:n]]

test_x = [data_x[i] for i in order[n:]]
test_y = [data_y[i] for i in order[n:]]

print(len(train_x))
print(len(test_y))

34200
30780
30780
3420


In [28]:
train_x_list, train_y_list = [], []
valid_x_list, valid_y_list = [], []

n=int(len(data_x)/10)
for i in range(0, 9):
#     train_x_list.append([train_x[i] for i in range(0, i*n)+range((i+1)*n), len(train_x)])
    cur_x = [train_x[i] for i in range(0, i*n)]
    cur_x.extend([train_x[i] for i in range((i+1)*n, len(train_x))])
    cur_y = [train_y[i] for i in range(0, i*n)]
    cur_y.extend([train_y[i] for i in range((i+1)*n, len(train_x))])
    print(len(cur_x))
#     print(len(cur_y))
    
    train_x_list.append(cur_x)
    train_y_list.append(cur_y)
    
    valid_x_list.append([train_x[i] for i in range(i*n, (i+1)*n)])
    valid_y_list.append([train_y[i] for i in range(i*n, (i+1)*n)])

print(len(train_x_list))

27360
27360
27360
27360
27360
27360
27360
27360
27360
9


In [32]:
#3-layer
graph = tf.Graph()
with graph.as_default(), tf.device('/gpu:0'):
    labels = tf.placeholder('float', [None, num_id])            #
    inputs = tf.placeholder('float', [None, img_h, img_w, img_c])                        #输入的数据占位符
#     print(x.shape)
#     print(y_actual.shape)

    def weight_variable(shape): #定义一个函数，用于初始化所有的权值 W
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape): #定义一个函数，用于初始化所有的偏置项 b
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W): #定义一个函数，用于构建卷积层
        return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='SAME')

    def max_pool(x): #定义一个函数，用于构建池化层
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

    #构建网络
#     x_image = tf.reshape(inputs, [-1,250,250,3])         #转换输入数据shape,以便于用于网络中
    W_conv1 = weight_variable([5, 5, 3, 32])      
    b_conv1 = bias_variable([32])       
    h_conv1 = tf.nn.relu(conv2d(inputs, W_conv1) + b_conv1)     #第一个卷积层
    h_pool1 = max_pool(h_conv1)                                  #第一个池化层
    print(h_pool1.shape)#(?, 124, 124, 32)

#     W_conv2 = weight_variable([5, 5, 32, 64])
#     b_conv2 = bias_variable([64])
#     h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)      #第二个卷积层
#     h_pool2 = max_pool(h_conv2)                                   #第二个池化层
#     print(h_pool2.shape)#(?, 30, 30, 64)
    
#     W_conv3 = weight_variable([5, 5, 64, 128])
#     b_conv3 = bias_variable([128])
#     h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)      #第二个卷积层
#     h_pool3 = max_pool(h_conv3)                                   #第二个池化层
#     print(h_pool3.shape)#(?, 30, 30, 64)    

    # 第三层 是个全连接层,输入维数125*125*64, 输出维数为1024
    W_fc1 = weight_variable([25 * 25 * 32, 512])
    b_fc1 = bias_variable([512])
    h_pool1_flat = tf.reshape(h_pool1, [-1, 25*25*32])              #reshape成向量
    h_fc1 = tf.nn.relu(tf.matmul(h_pool1_flat, W_fc1) + b_fc1)    #第一个全连接层

    keep_prob = tf.placeholder("float") 
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)                  #dropout层

    W_fc2 = weight_variable([512, num_id])
    b_fc2 = bias_variable([num_id])
#     y_predict=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)   #softmax层
    y_predict=tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#     loss = -tf.reduce_sum(labels*tf.log(y_predict))     #交叉熵
#     optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)    #梯度下降法
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_predict, labels = labels))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(labels,1))    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))    #精确度计算

(?, 25, 25, 32)


In [35]:
batch_start = 0
batch_size = 512
training_steps = 1000

for i in range(0, 9):
    train_x = train_x_list[i]
    train_y = train_y_list[i]
#     config = tf.ConfigProto(allow_soft_placement=True)
#     config.gpu_options.allocator_type = 'BFC'
#     config.gpu_options.per_process_gpu_memory_fraction = 0.8
    with tf.Session(graph=graph) as session:
    #     saver.restore(session, tf.train.latest_checkpoint('lstm_check'))
        tf.global_variables_initializer().run()
        print('Initialized', i)
        for step in range(training_steps): 
            if(batch_start+batch_size>len(train_x)):
                order = [i for i in range(len(train_x))]
                random.shuffle(order)
                train_x = [train_x[i] for i in order]
                train_y = [train_y[i] for i in order]
                batch_start = 0        
            feed_x = train_x[batch_start:batch_start+batch_size]
            feed_y = train_y[batch_start:batch_start+batch_size]
            feed_dict = {inputs: feed_x, labels: feed_y, keep_prob:0.6}
            _, l, predictions, accur = session.run([optimizer, loss, labels, accuracy], feed_dict = feed_dict)
            if (step % 10 == 0):
                print('Loss at step %d: %f' % (step, l), ' accuarcy is ', accur)
    #             saver.save(session, 'lstm_check/my-model-1.ckpt', global_step=step+1000)
        valid_loss, valid_accur = session.run([loss,accuracy], feed_dict={inputs: valid_x_list[i], labels:  valid_y_list[i], keep_prob:1.0})
        print('==valid loss, ', valid_loss, ' valid accur is ', valid_accur)
        test_loss, test_accur = session.run([loss,accuracy], feed_dict={inputs: test_x, labels: test_y, keep_prob:1.0})
        print('--test loss, ', test_loss, ' test accur is ', test_accur)

Initialized 0
Loss at step 0: 15.246866  accuarcy is  0.130859
Loss at step 10: 4.123201  accuarcy is  0.207031
Loss at step 20: 1.948706  accuarcy is  0.292969
Loss at step 30: 1.832580  accuarcy is  0.328125
Loss at step 40: 1.673771  accuarcy is  0.410156
Loss at step 50: 1.497019  accuarcy is  0.490234
Loss at step 60: 1.323831  accuarcy is  0.572266
Loss at step 70: 1.105918  accuarcy is  0.654297
Loss at step 80: 0.906679  accuarcy is  0.738281
Loss at step 90: 0.709028  accuarcy is  0.818359
Loss at step 100: 0.565327  accuarcy is  0.847656
Loss at step 110: 0.464086  accuarcy is  0.878906
Loss at step 120: 0.368504  accuarcy is  0.894531
Loss at step 130: 0.311233  accuarcy is  0.921875
Loss at step 140: 0.277879  accuarcy is  0.921875
Loss at step 150: 0.223323  accuarcy is  0.949219
Loss at step 160: 0.201858  accuarcy is  0.958984
Loss at step 170: 0.179473  accuarcy is  0.958984
Loss at step 180: 0.164740  accuarcy is  0.945312
Loss at step 190: 0.144722  accuarcy is  0.960

KeyboardInterrupt: 

9 - 0.35

5 - 0.3 - 0.4

3 - 0.55+